# LTCM Risk

In [1]:
import pandas as pd
import math

In [2]:
excess_return = pd.read_excel("hedge_data.xls", sheet_name=0)
equity_index = pd.read_excel("hedge_data.xls", sheet_name=2)
excess_return.head()

,date,gross,net
0,1994-04-30,0.0658,0.0508
1,1994-05-31,-0.0415,-0.0315
2,1994-06-30,0.1137,0.0817
3,1994-07-31,0.0355,0.0275
4,1994-08-31,-0.0061,-0.0051


### 1. Summary stats
#### a. Mean and volatility of excessive return

In [3]:
net_mean = excess_return['net'].mean()*12
net_vol = excess_return['net'].std()*math.sqrt(12)
gross_mean = excess_return['gross'].mean()*12
gross_vol = excess_return['gross'].std()*math.sqrt(12)

exret_mean_vol = pd.DataFrame({
    "mean": [net_mean, gross_mean],
    "volatility": [net_vol, gross_vol]
}, index=["net", "gross"])
exret_mean_vol

,mean,volatility
net,0.168918,0.113187
gross,0.257153,0.137987


#### b. Annualized Sharpe ratio

In [4]:
exret_mean_vol['mean']/exret_mean_vol['volatility']

net      1.492373
gross    1.863603
dtype: float64

The Sharpe ratio for the asset seems unexpectedly high. The net SR is about 300% higher than the average stock market SR $(0.39)$ and SRs of assets we saw in HW1 $(\leq0.34)$.

### 2. Estimate with LTCM regression model:
$$\tilde{r}^{LTCM}_t = \alpha + \beta^m \tilde{r}^{m}_t + \epsilon_t$$

In [5]:
import statsmodels.formula.api as smf

In [6]:
temp = pd.DataFrame({
    "net": excess_return['net'], 
    "market": equity_index[equity_index['date']<="1998-06-30"]['Market Equity Index']
})

results = smf.ols('net ~ market', data=temp).fit()
alpha = results.params[0]
beta_m = results.params[1]

print("Unadjusted R-square: {}\nAlpha: {}\nBeta: {}".format(results.rsquared, alpha, beta_m))

Unadjusted R-square: 0.0016513307508558839
Alpha: 0.013430806247735792
Beta: 0.04202792771598583


The low R-square shows that the not much variation in LTCM can be explained by the market index. Even if the regression holds, the beta indicate that, within the data's time range, movement of market equity index only correspond to 1.3% of movement in the return. Therefore, LTCM does not appear to have much correlation with the index factor.

### 3. Regression-based metrics

#### a. Treynor ratio

In [7]:
treynor = net_mean/beta_m
treynor

4.019176205886869

#### b. Information ratio

In [8]:
IR = alpha/results.resid.std()
IR

0.41139012789149837

### 4. Tail risk
#### a. 5th worst return of the sample.

In [9]:
sorted(excess_return['net'])[4]

-0.018099999999999998

#### b. Mean of the worst 4 returns from the sample.

In [10]:
sum(sorted(excess_return['net'])[:4])/4

-0.056575

#### c. Skewness of the return distribution

In [11]:
excess_return['net'].skew()

-0.8573139307755562

The skewness is smaller than 0, which means it is negatively skewed to the left.
#### d. Kurtosis of the return distribution

In [12]:
excess_return['net'].kurtosis()+3

5.987867188139747

The kurtosis is almost as twice higher as 3, meaning it has a significant amount of outliers/tail data comparing to a normal distribution.

## 3. Other Hedge Fund Indexes


### 1. Summary statistics

In [13]:
def report_stats(data):
    mean = data.mean()*12
    vol = data.std()*math.sqrt(12)
    skew = data.skew()
    kurt = data.kurtosis()+3
    fifth = sorted(data)[4]
    return [mean, vol, skew, kurt, fifth]

hedge_index = pd.read_excel("hedge_data.xls", sheet_name=1)

In [14]:
hedge = hedge_index[hedge_index['date']<="1998-06-30"]
hedge = hedge.drop('date', axis=1)

hedge_stats = pd.DataFrame(dict(hedge.apply(report_stats)), 
                           index=['Mean', "Volatile", "Skew", "Kurtosis", "Fifth Percentile"])
hedge_stats["Total Index"]

Mean                0.131929
Volatile            0.084846
Skew                0.174095
Kurtosis            3.168940
Fifth Percentile   -0.019600
Name: Total Index, dtype: float64

### 2. Regression of each series on the market-equity factor


In [15]:
MARKET = equity_index[equity_index['date']<="1998-06-30"]['Market Equity Index']

def run_regression(data):
    temp = pd.DataFrame({
        "net": data, 
        "market": MARKET
    })
    results = smf.ols('net ~ market', data=temp).fit()
    alpha = results.params[0]
    beta_m = results.params[1]
    r_square = results.rsquared
    return [alpha, beta_m, r_square]


hedge_regression = pd.DataFrame(dict(hedge.apply(run_regression)), 
                               index=['Alpha', "Beta", "R-square"])
hedge_regression

,Total Index,Convertible Arbitrage,Dedicated Short Bias,Emerging Markets,Equity Market Neutral,Event Driven,Event Driven Distressed,Event Driven Multi-Strategy,Event Driven Risk Arbitrage,Fixed Income Arbitrage,Global Macro,Long/Short Equity,Managed Futures,Multi-Strategy
Alpha,0.003888,0.003345,0.012182,-0.006820,0.002393,0.004265,0.005376,0.003857,0.002632,0.002770,0.008051,-0.000391,-0.002138,0.005408
Beta,0.462541,0.075847,-1.094786,0.615501,0.165458,0.236607,0.311428,0.171142,0.132719,0.056809,0.569501,0.621620,0.120503,-0.027706
R-square,0.355953,0.054253,0.636514,0.132392,0.237605,0.410441,0.391489,0.160367,0.187130,0.059413,0.207588,0.668906,0.013083,0.003414
